In [ ]:
import os
import torch
import cv2
import numpy as np

from ultralytics import SAM, YOLO


# SAM

In [13]:
# Configura el uso de CUDA
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [15]:
# Cargar SAM con el checkpoint
sam_model = SAM('Modelos/SAM/sam_b.pt')  

# Cargar el modelo YOLO
yolo_model = YOLO('Modelos/yolov10/HoldOutSimpleYolov10n.pt')

In [ ]:
# Directorios de imágenes y de salida
image_dir = "datasets/train/images"
output_dir = "datasets/SAM_output"  # Directorio para guardar las s egmentaciones
os.makedirs(output_dir, exist_ok=True)

In [20]:
def segment_stoma(image_path, output_dir, yolo_model, sam_model, clase, save_masks=False):
    masks_list = []
    image = cv2.imread(image_path)
    image_name = os.path.basename(image_path)
    
    results = yolo_model.predict(image, imgsz=640, device=device, verbose = False)

    for idx, detection in enumerate(results[0].boxes):
        if detection.cls == clase:
            x_min, y_min, x_max, y_max = map(int, detection.xyxy[0])

            sam_results = sam_model.predict(image, bboxes=[x_min, y_min, x_max, y_max], verbose = False)
            for mask in sam_results[0].masks.data:
                mask = mask.data.cpu().numpy()
                masks_list.append(mask)
                if save_masks:

                    mask_path = os.path.join(output_dir, f"{os.path.splitext(image_name)[0]}_mask_{idx}.png")
                    cv2.imwrite(mask_path, mask * 255)
    return masks_list

In [ ]:
def combine_masks(masks_list, image_shape):
    combined_mask = np.zeros(image_shape[:2], dtype=np.uint8)
    for mask in masks_list:
        combined_mask = np.maximum(combined_mask, mask)
    return combined_mask  

def overlay_combined_mask(original_image_path, combined_mask, output_path):
    original_image = cv2.imread(original_image_path)
    overlay = original_image.copy()
    overlay[combined_mask > 0] = [0, 255, 0]
    
    cv2.imwrite(output_path, overlay)

Testeo

In [ ]:
image_path = "datasets/train/images/STMHD0002.jpg"
output_path = "datasets/SAM_output/STMHD0002_mask_overlay.jpg"
clase = 0 # 0 es la caja grande, 1 es la caja pequeña

masks_list = segment_stoma(image_path, output_dir, yolo_model, sam_model, clase,  save_masks=False)
combined_mask = combine_masks(masks_list, cv2.imread(image_path).shape)

overlay_combined_mask(image_path, combined_mask, output_path)

In [32]:
# Si se quiere hacer para todos las imagenes 
image_dir = "datasets/train/images"
output_dir = "datasets/SAM_output"
clase = 1 # 0 es la caja grande, 1 es la caja pequeña
os.makedirs(output_dir, exist_ok=True)

for image_name in os.listdir(image_dir)[:2]:
    image_path = os.path.join(image_dir, image_name)
    masks_list = segment_stoma(image_path, output_dir, yolo_model, sam_model, clase, save_masks=False)
    combined_mask = combine_masks(masks_list, cv2.imread(image_path).shape)
    output_path = f"datasets/SAM_output/MaskOverlay_Class_{clase}_{image_name}"
    overlay_combined_mask(image_path, combined_mask, output_path)